Setup and Load Data

In [1]:
import tensorflow as tf
import os

In [2]:
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

Remove Dodgy images

In [3]:
import cv2

In [4]:
import imghdr

In [5]:
data_dir='C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\data\\'

In [6]:
os.listdir(data_dir)

['Happy People', 'Sad People']

In [7]:
image_exts=['jpeg','png','bmp','jpg']

In [8]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        path=os.path.join(data_dir,image_class)
        
        image_path=os.path.join(path,image)
        try:
            img=cv2.imread(image_path)
            tip=imghdr.what(image_path)
            if tip not in image_exts:
                print("Image not in exts list { }".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("issue with image {}".format(image_path))
            os.remove(image_path)

Load Data

In [9]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data=tf.keras.utils.image_dataset_from_directory(data_dir)

Found 209 files belonging to 2 classes.


In [11]:
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [12]:
data_iterator=data.as_numpy_iterator()

In [13]:
data_iterator

In [14]:
batch=data_iterator.next()

In [15]:
batch[1].shape

(32,)

Preprocess Data

In [16]:
data=data.map(lambda x,y:(x/255,y))

In [17]:
scaled_iterator=data.as_numpy_iterator()
batch=scaled_iterator.next()

Split Data

In [18]:
sz=len(data)
train_size=int(sz*0.7)
val_size=int(sz*0.2)+1
test_size=int(sz*0.1)+1

In [19]:
train=data.take(train_size)
val=data.skip(train_size).take(val_size)
test=data.skip(train_size+val_size).take(test_size)

In [20]:
len(test)

1

Deep Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [22]:
model=Sequential()

In [23]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [24]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

Train

In [26]:
logdir='logs'

In [27]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [28]:
hist=model.fit(train,epochs=20,validation_data=val,callbacks=[tensorboard_callback])

Epoch 1/20
4/4 [==============================] - 6s 1s/step - loss: 1.1797 - accuracy: 0.4531 - val_loss: 0.7056 - val_accuracy: 0.4531
Epoch 2/20
4/4 [==============================] - 3s 553ms/step - loss: 0.6515 - accuracy: 0.5469 - val_loss: 0.6449 - val_accuracy: 0.8438
Epoch 3/20
4/4 [==============================] - 3s 581ms/step - loss: 0.6265 - accuracy: 0.7734 - val_loss: 0.5400 - val_accuracy: 0.8750
Epoch 4/20
4/4 [==============================] - 3s 620ms/step - loss: 0.5684 - accuracy: 0.7266 - val_loss: 0.4710 - val_accuracy: 0.8750
Epoch 5/20
4/4 [==============================] - 3s 573ms/step - loss: 0.5234 - accuracy: 0.7812 - val_loss: 0.3916 - val_accuracy: 0.8594
Epoch 6/20
4/4 [==============================] - 3s 566ms/step - loss: 0.4094 - accuracy: 0.8984 - val_loss: 0.3695 - val_accuracy: 0.8750
Epoch 7/20
4/4 [==============================] - 3s 504ms/step - loss: 0.3149 - accuracy: 0.8594 - val_loss: 0.3146 - val_accuracy: 0.8438
Epoch 8/20
4/4 [=======

Evaluate and Performance

In [29]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

In [30]:
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [31]:
for batch in test.as_numpy_iterator():
    X,Y=batch
    yhat=model.predict(X)
    pre.update_state(Y,yhat)
    re.update_state(Y,yhat)
    acc.update_state(Y,yhat)

1/1 [==============================] - 0s 174ms/step


In [32]:
print(f'Precision:{pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}')

Precision:1.0,Recall:1.0,Accuracy:1.0


TEST

In [59]:
path="C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\240_F_671059717_OAlmXdOfyZiunLuzQqCjZ4FBwa0NZCel.jpg"

img=cv2.imread(path)
resize=tf.image.resize(img,(256,256))

In [60]:
np.expand_dims(resize,0).shape

(1, 256, 256, 3)

In [61]:
yhat=model.predict(np.expand_dims(resize/255,0))

1/1 [==============================] - 0s 44ms/step


In [62]:
yhat

array([[0.99998343]], dtype=float32)

In [63]:
if yhat<0.5:
    print("Predicted class is Happy")
else:
    print("Predicted class is Sad")

Predicted class is Sad


SAVE MODEL

In [64]:
from tensorflow.keras.models import load_model

In [65]:
model_path='C:\\Users\\91954\\OneDrive\\Desktop\\Models'

In [67]:
model.save(os.path.join(model_path,'HappySadModel.h5'))

In [68]:
new_model=load_model(os.path.join(model_path,'HappySadModel.h5'))

In [69]:
yhatn=new_model.predict(np.expand_dims(resize/255,0))

1/1 [==============================] - 0s 184ms/step


In [70]:
yhatn

array([[0.99998343]], dtype=float32)

In [71]:
if yhatn<0.5:
    print("Predicted class is Happy")
else:
    print("Predicted class is Sad")

Predicted class is Sad
